In [1]:
import chardet
import openai  # dependency problem: python -m pip install charset-normalizer==2.1.0
import os
import pandas as pd
import numpy as np

from features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor import extract_knowledge_from_df
from dotenv import load_dotenv

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Agentic knowledge extractor

This notebook uses a Large Language Model (LLM) as an agent in order to extract knowledge from text input. The text input are the results from the fuzzy search, which returns the keyword found embedded in its context (words before and after the keyword). In fact, we are not interested in the keyword itself, but in its actual value. For example, the keyword may be "Firsthöhe" (= ridge height), so we'd like to find out the actual height. 

## Get API key

To be able to use the GPT API, add a file to the project root directory titled `.env` and add your API key to it: `OPENAI_API_KEY=[KEY]`

In [2]:
load_dotenv()
openai.api_key=os.getenv('OPENAI_API_KEY')

## Set variables

In [5]:
INPUT_FOLDER_PATH = os.path.join("..", "data", "nrw", "features", "fuzzy_search", "traufhöhe_80.csv")

ID_COLUMN='filename'
TEXT_COLUMN='content'

In [6]:
with open(INPUT_FOLDER_PATH, 'rb') as f:
    enc = chardet.detect(f.read())

input_df = pd.read_csv(INPUT_FOLDER_PATH,
                       names=[ID_COLUMN, TEXT_COLUMN],
                       encoding = enc['encoding'], header=None)[10:20]

## Specify keyword dictionary

Change the keyword _values_ (on the right hand side), if necessary.

In [ ]:
keyword_dict = {
    'keyword':          'traufhöhe',
    'keyword_short':    'th',
    'template_name':    'features/textual_features/knowledge_extraction_agent/prompt_templates/knowledge_extraction_th.template',
    'filename':         'traufhöhe.csv'
    }

In [ ]:
all_responses = extract_knowledge_from_df(keyword_dict=keyword_dict,
                                          input_df=input_df,
                                          id_column_name=ID_COLUMN,
                                          text_column_name=TEXT_COLUMN)

2023-09-14 14:21:38.812 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:146 - Relevant keyword(s): traufhöhe
2023-09-14 14:21:38.813 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:147 - Extracting relevant info from text snippets via LLM agent.
2023-09-14 14:21:50.334 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:165 - Info extracted. Extracting numerical value from info.
2023-09-14 14:21:50.338 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:171 - Numerical values extracted. Validating their occurrence in input text.
2023-09-14 14:21:50.342 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:180 - Returning results for th.


In [ ]:
all_responses.head(10)

,id,th_input,th_agent_response,th_extracted_value,validation
10,2220412_1.pdf,"nutzung (§ 9 abs. 1 nr. 1 baugb) 2.1 höhe baulicher anlagen (§ 18 baunvo) 2.1.1 trauf und firsthöhe die traufhöhe wird definiert als schnittpunkt der verlängerung der außenkante der außenwand mit der dachhaut (pfannenlage). als firsthöhe gilt der oberste dachabschluss ;;; diejenige straße zugrunde gelegt, von der aus die haupterschließung des gebäudes erfolgt. 2.1.3 bei flachdächern gilt das maß für die traufhöhe als maximale gebäudehöhe. 2.1.4 werden pultdächer errichtet, darf die traufhöhe für die höhere wand um 1,5 m erhöht werden. 3 ;;; gebäudes erfolgt. 2.1.3 bei flachdächern gilt das maß für die traufhöhe als maximale gebäudehöhe. 2.1.4 werden pultdächer errichtet, darf die traufhöhe für die höhere wand um 1,5 m erhöht werden. 3 garagen, carports und stellplätze (§ 9 abs. 1 nr. 4 ;;; als straelen, den 24.4.18 entwurf und bearbeitung: in vertretung: purath gez. purath stadt straelen der bürgermeister essen, den 23.4.18 maximale traufhöhe in meter über th max. maximale firsthöhe in meter über fh max. bezugspunkt bezugspunkt versickerungsanlagen sind mit dem baugesuch der",th: None,NaN,True
11,2220438_13.pdf,"flurgrenze stadt straelen ge abwasserbeseitigung 286 gewerbegebiete firsthöhe gebäude geschoßflächenzahl mass der baul. nutzungart der baulichen nutzung flurstücksgrenze ogemischte bauflächen traufhöhe 1,8 fh grz th a baugrenze aufteilung als hinweis bauweise, baugrenzen offene bauweise abweichende bauweise straßenbegrenzungslinie straßenverkehrsfläche verkehrsflächen natur und ;;; vor geschriebenen veröffentlichung tritt, tritt der straelen, den 25.10.2000 gez. giesen bürgermeister ,ggfls., werbeanlagen sind nur an gebäudeteilen unterhalb der traufhöhe (auch an giebeln) zulässig. an einfriedungen und auf dachflächen sind sie nicht zulässig. je gewerbebetrieb ist die firmenbezeichnung als beleuchtete","th: 1,8 m",1.80,True
12,2220449_0.pdf,maß der baulichen nutzung das maß der baulichen nutzung wird im vorliegenden bebauungsplan durch die grundflächenzahl (grz) und die maximale trauhöhe (th max.) und firsthöhe (fh max.) bestimmt. grundflächenzahl (grz) das höchstmaß der baulichen nutzung wird für die allgemeinen wohngebiete mit,th max.: 10 m\nth max.: 12 m\nth max.: 15 m\nth max.: 10 m\nth max.: 10 m\nth max.: 10,NaN,True
13,2220497_3.pdf,"baulichen nutzung (§ 9 abs. 1 nr. 1 baugb, §§ 16 bis 21a baunvo) grundflächenzahl als höchstmaß 0,4 wr maximale traufhöhe in meternth max anzahl vollgeschosse i 3. bauweise, baulinien, baugrenzen (§ 9 abs. 1 nr. 2 baugb, § 22 und ;;; baulichen nutzung (§ 9 abs. 1 nr. 1 baugb) höhe baulicher anlagen (§ 18 baunvo) oberer bezugspunkt zur ermittlung der traufhöhe (th max) ist der schnittpunkt der außenwand des gebäudes mit der oberkante der dachhaut (pfannenlage). unterer bezugspunkt zur ermittlung der ;;; (th max) ist der schnittpunkt der außenwand des gebäudes mit der oberkante der dachhaut (pfannenlage). unterer bezugspunkt zur ermittlung der traufhöhe (th max) ist jeweils die gemittelte höhe der fertig ausgebauten, an das plangebiet angrenzenden uhlandstraße an der gebäudeseitigen straßenbegrenzungslinie zwischen",th: None,NaN,True
14,2220497_4.pdf,"der bebauungsplan den festsetzungen des ehemaligen bebauungsplanes nr. 5 und entspricht ebenso der derzeitigen bestandssituation. 7.4 höhe baulicher anlagen die traufhöhe für das eingeschossige gebäude wird mit maximal 4,75 m über der an grenzenden vorhandenen erschließungsstraße festgesetzt. damit wird gewährleistet, dass","th: 4,75 m",4.75,True
15,2220502_13.pdf,"der baulichen nutzung (§ 9 abs. 1 baugb, 2. abschnitt baunvo) höhe baulicher anlagen (§ 18 baunvo) das maß der traufhöhe stellt den schnittpunkt zwischen der senkrechten außenfläche und der dachhaut dar. das maß der firsthöhe stellt den obersten dachabschluss dar.",th: None,NaN,True
16,2220502_15.pdf,"flachdächer zulässig. 6.3 zusammenhängende garagen sind im gleichem material, in gleicher farbe und 

## Reset if validation unsuccessful

If validation wasn't successful (= False), the extracted_value column should be set back to NaN.

In [ ]:
column_name = f"{keyword_dict['keyword_short']}_extracted_value"
all_responses.loc[all_responses['validation'] == False, column_name] = np.nan

all_responses.head()

,id,th_input,th_agent_response,th_extracted_value,validation
10,2220412_1.pdf,"nutzung (§ 9 abs. 1 nr. 1 baugb) 2.1 höhe baulicher anlagen (§ 18 baunvo) 2.1.1 trauf und firsthöhe die traufhöhe wird definiert als schnittpunkt der verlängerung der außenkante der außenwand mit der dachhaut (pfannenlage). als firsthöhe gilt der oberste dachabschluss ;;; diejenige straße zugrunde gelegt, von der aus die haupterschließung des gebäudes erfolgt. 2.1.3 bei flachdächern gilt das maß für die traufhöhe als maximale gebäudehöhe. 2.1.4 werden pultdächer errichtet, darf die traufhöhe für die höhere wand um 1,5 m erhöht werden. 3 ;;; gebäudes erfolgt. 2.1.3 bei flachdächern gilt das maß für die traufhöhe als maximale gebäudehöhe. 2.1.4 werden pultdächer errichtet, darf die traufhöhe für die höhere wand um 1,5 m erhöht werden. 3 garagen, carports und stellplätze (§ 9 abs. 1 nr. 4 ;;; als straelen, den 24.4.18 entwurf und bearbeitung: in vertretung: purath gez. purath stadt straelen der bürgermeister essen, den 23.4.18 maximale traufhöhe in meter über th max. maximale firsthöhe in meter über fh max. bezugspunkt bezugspunkt versickerungsanlagen sind mit dem baugesuch der",th: None,NaN,True
11,2220438_13.pdf,"flurgrenze stadt straelen ge abwasserbeseitigung 286 gewerbegebiete firsthöhe gebäude geschoßflächenzahl mass der baul. nutzungart der baulichen nutzung flurstücksgrenze ogemischte bauflächen traufhöhe 1,8 fh grz th a baugrenze aufteilung als hinweis bauweise, baugrenzen offene bauweise abweichende bauweise straßenbegrenzungslinie straßenverkehrsfläche verkehrsflächen natur und ;;; vor geschriebenen veröffentlichung tritt, tritt der straelen, den 25.10.2000 gez. giesen bürgermeister ,ggfls., werbeanlagen sind nur an gebäudeteilen unterhalb der traufhöhe (auch an giebeln) zulässig. an einfriedungen und auf dachflächen sind sie nicht zulässig. je gewerbebetrieb ist die firmenbezeichnung als beleuchtete","th: 1,8 m",1.80,True
12,2220449_0.pdf,maß der baulichen nutzung das maß der baulichen nutzung wird im vorliegenden bebauungsplan durch die grundflächenzahl (grz) und die maximale trauhöhe (th max.) und firsthöhe (fh max.) bestimmt. grundflächenzahl (grz) das höchstmaß der baulichen nutzung wird für die allgemeinen wohngebiete mit,th max.: 10 m\nth max.: 12 m\nth max.: 15 m\nth max.: 10 m\nth max.: 10 m\nth max.: 10,NaN,True
13,2220497_3.pdf,"baulichen nutzung (§ 9 abs. 1 nr. 1 baugb, §§ 16 bis 21a baunvo) grundflächenzahl als höchstmaß 0,4 wr maximale traufhöhe in meternth max anzahl vollgeschosse i 3. bauweise, baulinien, baugrenzen (§ 9 abs. 1 nr. 2 baugb, § 22 und ;;; baulichen nutzung (§ 9 abs. 1 nr. 1 baugb) höhe baulicher anlagen (§ 18 baunvo) oberer bezugspunkt zur ermittlung der traufhöhe (th max) ist der schnittpunkt der außenwand des gebäudes mit der oberkante der dachhaut (pfannenlage). unterer bezugspunkt zur ermittlung der ;;; (th max) ist der schnittpunkt der außenwand des gebäudes mit der oberkante der dachhaut (pfannenlage). unterer bezugspunkt zur ermittlung der traufhöhe (th max) ist jeweils die gemittelte höhe der fertig ausgebauten, an das plangebiet angrenzenden uhlandstraße an der gebäudeseitigen straßenbegrenzungslinie zwischen",th: None,NaN,True
14,2220497_4.pdf,"der bebauungsplan den festsetzungen des ehemaligen bebauungsplanes nr. 5 und entspricht ebenso der derzeitigen bestandssituation. 7.4 höhe baulicher anlagen die traufhöhe für das eingeschossige gebäude wird mit maximal 4,75 m über der an grenzenden vorhandenen erschließungsstraße festgesetzt. damit wird gewährleistet, dass","th: 4,75 m",4.75,True
